In [8]:
import time

from konlpy.tag import Mecab
from datasketch import MinHash, MinHashLSH

# 전역 Mecab 객체 생성
mecab = Mecab()

In [9]:
def get_minhash(text, num_perm=128):
    """
    주어진 텍스트에 대해 MinHash 객체를 생성합니다.
    """
    tokens = mecab.morphs(text)
    m = MinHash(num_perm=num_perm)
    for token in tokens:
        m.update(token.encode('utf8'))
    return m

def deduplicate_texts_lsh(texts, threshold=0.8, num_perm=128):
    """
    LSH 알고리즘을 사용하여 유사한 한국어 텍스트를 deduplicate 합니다.
    
    Parameters:
        texts (list): 중복 제거할 텍스트 리스트.
        threshold (float): LSH threshold (0과 1 사이, 높을수록 더 유사한 텍스트끼리 그룹화).
        num_perm (int): MinHash의 permutation 수.
        
    Returns:
        list: deduplicate 된 텍스트 리스트.
    """
    # LSH 객체 생성
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    minhashes = {}
    
    # 각 텍스트에 대해 MinHash를 생성하고 LSH에 삽입
    for i, text in enumerate(texts):
        m = get_minhash(text, num_perm)
        key = f"doc_{i}"
        lsh.insert(key, m)
        minhashes[key] = (text, m)
    
    seen = set()
    deduped = []
    
    # LSH를 통해 유사한 텍스트 그룹을 찾고, 그룹 내에서 대표 텍스트 하나만 선택
    for key, (text, m) in minhashes.items():
        if key in seen:
            continue
        similar_keys = lsh.query(m)
        for sim_key in similar_keys:
            seen.add(sim_key)
        deduped.append(text)
        
    return deduped

In [11]:
strings = [
    "안녕하세요. 저는 학생입니다.",
    "안녕하세요 저는 학생입니다",
    "반갑습니다. 저는 학생이에요.",
    "안녕하세요, 학생입니다.",
    "이것은 다른 문장입니다."
]
strings = [
    "hello"*1000,
    "hallo"*1000,
    "hello"*1000,
    "world"*1000,
    "wurld"*1000,
    "worlds"*1000
]



start = time.time()
result = deduplicate_texts_lsh(strings, threshold=0.8)
end = time.time()
print(f"{end-start:.3f}",len(result))

for text in result:
    print(text)

0.294 5
...
